In [1]:
'''
Objectives [/]:

1. Get Market Type [/]
2. Get Date [/]
3. Get Center [/]
4. Get Data [/]
    i.    Get id
    ii.   Get Variety Name
    iii.  Get Grade
    iv. Get Unit (Kilogram => KG)
    v.  Get Min Price
    vi.   Get Avg Price
    vii.  Get Max Price
5. Convert into dataframe and then to CSV[/]

'''

'\nObjectives [/]:\n\n1. Get Market Type [/]\n2. Get Date [/]\n3. Get Center [/]\n4. Get Data [/]\n    i.    Get id\n    ii.   Get Variety Name\n    iii.  Get Grade\n    iv. Get Unit (Kilogram => KG)\n    v.  Get Min Price\n    vi.   Get Avg Price\n    vii.  Get Max Price\n5. Convert into dataframe and then to CSV[/]\n\n'

In [2]:
import re
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
service = Service(executable_path=ChromeDriverManager().install())
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(service=service, options=op)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [4]:
market_type_regex = "(Farm|Wholesale|Retail)\sPrice"
date_regex = "[\d]{2}\s[A-Z]+\s[\d]{4}"

In [5]:
base_URL = "https://www.fama.gov.my/en/harga-pasaran-terkini"

driver.get(base_URL)
main_soup = BeautifulSoup(driver.page_source, "html.parser")

# main_page = requests.get(base_URL)
# main_soup = BeautifulSoup(main_page.content, "html.parser")

market_price_type = dict()
market_price_type_tag = main_soup.find_all("a", string=re.compile(market_type_regex))

for market_type in market_price_type_tag:
    market_price_type[market_type.text] = market_type.get("href")

In [6]:
def get_centres(soup):
    centres = list()
    for b_element in soup.find_all('b'):
        if b_element.text not in ["[", "]"]:
            centre_name = " ".join(b_element.text.split(" ")[-3:]).strip(" :")
            centres.append(centre_name)
    return centres

In [7]:
def get_data(soup, centres, market):
    rows = list()
    data_tables = soup.find_all('table')[0].find_all('table')[1::2]
    for tables, centre in zip(data_tables, centres):
        for tr_element in tables.find_all('tr')[2:]:
            td_elements = tr_element.find_all('td')
            # variety_id = tr_element['id']
            variety_name = td_elements[0].text
            grade = td_elements[1].text
            unit = td_elements[2].text.replace("KILOGRAM", "KG")
            max_price = "%.2f" % float(td_elements[3].text)
            avg_price = "%.2f" % float(td_elements[4].text)
            min_price =  "%.2f" % float(td_elements[5].text)
            rows.append([market, centre, variety_name, grade, unit, max_price, avg_price, min_price])
    return rows

In [8]:
data = list()

for market in market_price_type:
    driver.get(market_price_type[market])
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Get date uploaded
    date = re.search(date_regex, soup.find('h3').text).group(0)

    # Get centers list
    centres = get_centres(soup)
    
    # Get data
    data = data + get_data(soup, centres, market)

In [9]:
len(data)

1663

In [10]:
# Get the centre
df = pd.DataFrame(data, columns = ['Market Price Type', 'Centre', 'Variety Name', 'Grade', 'Unit', 'Max Price', 'Average Price', 'Min. Price'])

In [11]:
df.to_csv("{}_market_price.csv".format("_".join(date.lower().split(" "))), index=False)

In [12]:
df

,Market Price Type,Centre,Variety Name,Grade,Unit,Max Price,Average Price,Min. Price
0,Farm Price,"JOHOR BAHRU, JOHOR",AYAM HIDUP,F.A.Q,100 KG,610.00,610.00,610.00
1,Farm Price,"JOHOR BAHRU, JOHOR",BAYAM,F.A.Q,100 KG,400.00,350.00,300.00
2,Farm Price,"JOHOR BAHRU, JOHOR",BETIK SEKAKI,F.A.Q,100 KG,200.00,190.00,180.00
3,Farm Price,"JOHOR BAHRU, JOHOR",BUNGA KANTAN,F.A.Q,TANGKAI,0.50,0.50,0.50
4,Farm Price,"JOHOR BAHRU, JOHOR",CILI HIJAU,F.A.Q,100 KG,800.00,750.00,700.00
...,...,...,...,...,...,...,...,...
1658,Retail Price,"LABUAN, W.P. LABUAN",TEMBIKAI MERAH TANPA BIJI,F.A.Q,KG,4.00,3.75,3.50
1659,Retail Price,"LABUAN, W.P. LABUAN",TERUNG PANJANG / PEPARU (TANAH RENDAH),F.A.Q,KG,7.00,6.75,6.50
1660,Retail Price,"LABUAN, W.P. LABUAN",TIMUN HIJAU,F.A.Q,KG,5.00,4.65,4.30
1661,Retail Price,"LABUAN, W.P. LABUAN",TOMATO (TANAH TINGGI),F.A.Q,KG,10.00,9.50,9.00
